In [1]:
import pandas as pd
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import requests
import plotly.express as px
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)
pd.set_option('display.float_format', lambda x: f'%.{2}f' % x)
pd.set_option('display.max_colwidth', None)

In [2]:
df = pd.read_csv('../8Jul2020-14Jul2020.csv')
df['Date'] = pd.to_datetime(df['Start Date'], dayfirst=True)
df.sort_values(by='Date', inplace=True)
df['Date'] = df['Date'].dt.date
df.head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,EndStation lat,EndStation long,EndStation nbBikes,EndStation nbStandardBikes,EndStation nbEBikes,EndStation nbEmptyDocks,EndStation nbDocks,StartStation lat,StartStation long,StartStation nbBikes,StartStation nbStandardBikes,StartStation nbEBikes,StartStation nbEmptyDocks,StartStation nbDocks,Date
0,99188805,780,13656,08/07/2020 00:13,476,"Stebondale Street, Cubitt Town",08/07/2020 00:00,532,"Jubilee Plaza, Canary Wharf",51.49,-0.01,26,24,2,3,29,51.50,-0.02,46,45,1,17,63,2020-07-08
1,99188806,180,4393,08/07/2020 00:03,531,"Twig Folly Bridge, Mile End",08/07/2020 00:00,517,"Ford Road, Old Ford",51.53,-0.04,6,6,0,15,21,51.53,-0.03,2,2,0,16,18,2020-07-08
2,99188809,960,8973,08/07/2020 00:17,508,"Fournier Street, Whitechapel",08/07/2020 00:01,779,"Houndsditch, Aldgate",51.52,-0.07,8,8,0,10,18,51.51,-0.08,3,3,0,25,28,2020-07-08
3,99188810,960,7141,08/07/2020 00:17,508,"Fournier Street, Whitechapel",08/07/2020 00:01,779,"Houndsditch, Aldgate",51.52,-0.07,8,8,0,10,18,51.51,-0.08,3,3,0,25,28,2020-07-08
4,99188807,600,15085,08/07/2020 00:11,201,"Dorset Square, Marylebone",08/07/2020 00:01,797,"Ossulston Street, Somers Town",51.52,-0.16,15,15,0,1,16,51.53,-0.13,28,27,1,2,30,2020-07-08


In [3]:
temp = df.groupby(['Date', 'EndStation Name'])['EndStation nbBikes'].agg(['first', 'last']).diff(axis=1)
# temp = df.groupby(['Date', 'EndStation Name'])['EndStation nbBikes'].agg(['first', 'last'])
temp.reset_index(drop=False, inplace=True)

# temp.drop(['first'], inplace=True, axis=1)
# temp.rename(columns = {'last':'Diff'}, inplace = True)
temp.head()

,Date,EndStation Name,first,last
0,2020-07-08,"Abbey Orchard Street, Westminster",NaN,0
1,2020-07-08,"Abbotsbury Road, Holland Park",NaN,0
2,2020-07-08,"Aberdeen Place, St. John's Wood",NaN,0
3,2020-07-08,"Aberfeldy Street, Poplar",NaN,0
4,2020-07-08,"Abingdon Green, Westminster",NaN,0


In [4]:
temp.describe()

,first,last
count,0.00,5342.00
mean,NaN,0.00
std,NaN,0.00
min,NaN,0.00
25%,NaN,0.00
50%,NaN,0.00
75%,NaN,0.00
max,NaN,0.00


In [5]:
temp = df.groupby(['Date', 'StartStation Name'])['Start Date','StartStation nbBikes'].agg(['first', 'last'])
temp.reset_index(drop=False, inplace=True)
temp.head()

C:\Users\rahin\AppData\Local\Temp\ipykernel_30712\478296273.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  temp = df.groupby(['Date', 'StartStation Name'])['Start Date','StartStation nbBikes'].agg(['first', 'last'])


Date                  StartStation Name        Start Date  \
                                                             first   
0  2020-07-08  Abbey Orchard Street, Westminster  08/07/2020 06:56   
1  2020-07-08      Abbotsbury Road, Holland Park  08/07/2020 11:01   
2  2020-07-08    Aberdeen Place, St. John's Wood  08/07/2020 05:59   
3  2020-07-08           Aberfeldy Street, Poplar  08/07/2020 09:03   
4  2020-07-08        Abingdon Green, Westminster  08/07/2020 08:21   

                    StartStation nbBikes       
               last                first last  
0  08/07/2020 21:32                   15   15  
1  08/07/2020 19:36                    8    8  
2  08/07/2020 18:49                    1    1  
3  08/07/2020 19:31                   12   12  
4  08/07/2020 23:55                   14   14

In [6]:
temp[temp['StartStation Name'] == 'Shoreditch High Street, Shoreditch']

Date                   StartStation Name        Start Date  \
                                                                 first   
591   2020-07-08  Shoreditch High Street, Shoreditch  08/07/2020 09:34   
1353  2020-07-09  Shoreditch High Street, Shoreditch  09/07/2020 01:00   
2116  2020-07-10  Shoreditch High Street, Shoreditch  10/07/2020 02:14   
2880  2020-07-11  Shoreditch High Street, Shoreditch  11/07/2020 00:00   
3644  2020-07-12  Shoreditch High Street, Shoreditch  12/07/2020 00:10   
4408  2020-07-13  Shoreditch High Street, Shoreditch  13/07/2020 00:22   
5171  2020-07-14  Shoreditch High Street, Shoreditch  14/07/2020 02:30   

                       StartStation nbBikes       
                  last                first last  
591   08/07/2020 23:46                   24   24  
1353  09/07/2020 23:34                   24   24  
2116  10/07/2020 23:19                   24   24  
2880  11/07/2020 23:57                   24   24  
3644  12/07/2020 23:30                   24   24  
4408  13/07/2020 23:57                   24   24  
5171  14/07/2020 23:16                   24   24

In [7]:
temp = df.groupby(['EndStation Name'])['EndStation nbBikes'].agg(['first', 'last']).diff(axis=1)
temp.reset_index(drop=False, inplace=True)

temp.drop(['first'], inplace=True, axis=1)
temp.rename(columns = {'last':'Diff'}, inplace = True)
temp.describe()

,Diff
count,764.00
mean,0.00
std,0.00
min,0.00
25%,0.00
50%,0.00
75%,0.00
max,0.00
